# Task 1: Data Exploration and Enrichment

## Objective
The goal of this notebook is to load the starter dataset, understand its structure, identify data gaps (specifically between Global Findex survey years), and enrich it with external high-frequency data (e.g., EthioTelecom reports, NBE directives) to prepare for forecasting.

## 1. Environment Setup & Data Loading
We load the unified dataset and reference codes using our custom `loader` module.

In [1]:

import sys
import os
import pandas as pd

# Add src to path so we can import loader
sys.path.append(os.path.abspath('..'))
from src.loader import load_data, load_reference_codes


In [ ]:
# Load data
df = load_data('../data/raw/ethiopia_fi_unified_data.xlsx')
refs = load_reference_codes('../data/raw/reference_codes.xlsx')

Total records: 43
Record Types: ['observation' 'target' 'event']
Years covered: [2014 2017 2021 2024 2023 2025]
indicator_code
ACC_OWNERSHIP         6
ACC_FAYDA             3
ACC_4G_COV            2
USG_P2P_COUNT         2
GEN_GAP_ACC           2
ACC_MM_ACCOUNT        2
USG_MPESA_USERS       1
GEN_MM_SHARE          1
AFF_DATA_INCOME       1
USG_ACTIVE_RATE       1
USG_MPESA_ACTIVE      1
USG_CROSSOVER         1
USG_TELEBIRR_VALUE    1
USG_TELEBIRR_USERS    1
USG_ATM_VALUE         1
USG_ATM_COUNT         1
USG_P2P_VALUE         1
ACC_MOBILE_PEN        1
GEN_GAP_MOBILE        1
Name: count, dtype: int64


## 2. Initial Data Audit
We examine the structure of the provided data to understand:
1.  **Temporal Coverage:** Do we have data for every year, or are there gaps?
2.  **Indicator Availability:** Which metrics (Access vs. Usage) have the most data points?
3.  **Data Types:** Distinguishing between *Observations* (measured values) and *Events* (contextual milestones).

In [ ]:
# 1. Quick Look
print(f"Total records: {len(df)}")
print(f"Record Types: {df['record_type'].unique()}")

# 2. Check Temporal Coverage (Years)
observations = df[df['record_type'] == 'observation']
print("Years covered:", observations['observation_date'].dt.year.unique())

# 3. Identify Gaps
# Check which indicators we have
print(observations['indicator_code'].value_counts())

Total records: 43
Record Types: ['observation' 'target' 'event']
Years covered: [2014 2017 2021 2024 2023 2025]


### 💡 Gap Analysis
**Observation:** The core Findex data (Account Ownership) is only available for **2014, 2017, 2021, and 2024**.
**Problem:** To forecast 2025-2027 accurately, we cannot rely on 4 data points over 10 years.
**Solution:** We need to enrich the dataset with annual proxy indicators that drive financial inclusion:
*   **Infrastructure:** Mobile Penetration (DataReportal/ITU) - The physical "rails" for digital finance.
*   **Usage Proxies:** Telebirr Subscriber counts (EthioTelecom) - High-frequency data showing the adoption curve between 2021-2024.
*   **Events:** Specific dates for market disruptions (e.g., M-Pesa launch).

## 3. Data Enrichment
In this section, we programmatically add external data to the dataset.

**Data Sources Added:**
1.  **DataReportal (2023-2024):** Mobile Penetration rates. High correlation with digital payment access.
2.  **EthioTelecom Annual Reports (2023-2024):** Telebirr registered user counts. This fills the usage gap between Findex 2021 and 2024.
3.  **Safaricom Ethiopia:** Precise date of M-Pesa's market entry.

*Note: We map these external sources to the unified schema, matching columns like `pillar`, `indicator_code`, and `record_type`.*

In [ ]:
# ---------------------------------------------------------
# REAL DATA ENRICHMENT
# Sources: EthioTelecom Annual Reports, DataReportal, World Bank
# ---------------------------------------------------------

real_enrichment_data = [
    # --- OBSERVATIONS: INFRASTRUCTURE (The "Rails" for Fintech) ---
    {
        "record_type": "observation",
        "pillar": "Infrastructure",
        "indicator": "Mobile Penetration",
        "indicator_code": "ACC_MOBILE_PEN", 
        "value_numeric": 53.5,
        "value_type": "percentage",
        "observation_date": pd.Timestamp("2023-01-01"),
        "source_name": "DataReportal Digital 2023",
        "source_url": "https://datareportal.com/reports/digital-2023-ethiopia",
        "confidence": "High",
        "notes": "66.8 million connections"
    },
    {
        "record_type": "observation",
        "pillar": "Infrastructure",
        "indicator": "Mobile Penetration",
        "indicator_code": "ACC_MOBILE_PEN", 
        "value_numeric": 60.4,
        "value_type": "percentage",
        "observation_date": pd.Timestamp("2024-01-01"),
        "source_name": "DataReportal Digital 2024",
        "source_url": "https://datareportal.com/reports/digital-2024-ethiopia",
        "confidence": "High",
        "notes": "77.4 million connections (+12% YoY)"
    },
    
    # --- OBSERVATIONS: USAGE (Telebirr Actuals) ---
    {
        "record_type": "observation",
        "pillar": "Usage",
        "indicator": "Telebirr Registered Users",
        "indicator_code": "USG_TELEBIRR_USERS", 
        "value_numeric": 27200000, # 27.2 Million
        "value_type": "count",
        "observation_date": pd.Timestamp("2023-01-01"),
        "source_name": "EthioTelecom H1 Report",
        "confidence": "High",
        "notes": "27.2M users reported in Jan 2023"
    },
    {
        "record_type": "observation",
        "pillar": "Usage",
        "indicator": "Telebirr Registered Users",
        "indicator_code": "USG_TELEBIRR_USERS", 
        "value_numeric": 47550000, # 47.55 Million
        "value_type": "count",
        "observation_date": pd.Timestamp("2024-06-30"),
        "source_name": "EthioTelecom Annual Report 2023/24",
        "confidence": "High",
        "notes": "Massive growth to 47.55M users"
    },

    # --- EVENTS: MAJOR MARKET SHIFTS ---
    {
        "record_type": "event",
        "category": "Product Launch",
        "pillar": "Infrastructure",
        "event_name": "Safaricom M-Pesa Launch",
        "indicator_code": "USG_MPESA_USERS", # Linked indicator
        "observation_date": pd.Timestamp("2023-08-16"),
        "source_name": "Safaricom Press Release",
        "notes": "M-Pesa goes live, introducing first major competition to Telebirr"
    },
    {
        "record_type": "event",
        "category": "Policy",
        "pillar": "Access",
        "event_name": "NBE Mobile Money Directive Revised",
        "indicator_code": "ACC_MM_ACCOUNT",
        "observation_date": pd.Timestamp("2023-10-01"), # Approx date of effective policy shift
        "source_name": "National Bank of Ethiopia",
        "notes": "Raised transaction limits, encouraging higher usage"
    }
]

# Convert to DataFrame
new_data_df = pd.DataFrame(real_enrichment_data)

# Align columns (fill missing columns with NaN to match schema)
for col in df.columns:
    if col not in new_data_df.columns:
        new_data_df[col] = pd.NA

# Reorder columns to match original
new_data_df = new_data_df[df.columns]

# Concatenate
enriched_df = pd.concat([df, new_data_df], ignore_index=True)

print(f"Original Row Count: {len(df)}")
print(f"Enriched Row Count: {len(enriched_df)}")

Original Row Count: 43
Enriched Row Count: 49


C:\Users\natty\AppData\Local\Temp\ipykernel_10168\3997254264.py:96: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  enriched_df = pd.concat([df, new_data_df], ignore_index=True)
C:\Users\natty\AppData\Local\Temp\ipykernel_10168\3997254264.py:96: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  enriched_df = pd.concat([df, new_data_df], ignore_index=True)


## 4. Verification & Export
We verify that the new rows have been added correctly and align with the existing schema before saving the enriched dataset to the `processed/` directory.

In [16]:
# Check the new "Access" data points
print("\n--- New Data Verification ---")
display(enriched_df[enriched_df['source_name'].str.contains("DataReportal")][['pillar', 'indicator_code', 'value_numeric', 'observation_date']])

# Save to processed
output_path = '../data/processed/ethiopia_fi_enriched.csv'
enriched_df.to_csv(output_path, index=False)
print(f"\n✅ Success! Enriched data saved to {output_path}")


--- New Data Verification ---


,pillar,indicator_code,value_numeric,observation_date
10,ACCESS,ACC_MOBILE_PEN,61.4,2025-12-31
43,Infrastructure,ACC_MOBILE_PEN,53.5,2023-01-01
44,Infrastructure,ACC_MOBILE_PEN,60.4,2024-01-01



✅ Success! Enriched data saved to ../data/processed/ethiopia_fi_enriched.csv


# Summary of Task 1
**Status:** Complete ✅

We successfully transformed the raw dataset. The file `data/processed/ethiopia_fi_enriched.csv` now contains:
*   **Original Data:** Findex surveys and targets.
*   **New Observations:** Mobile penetration (53.5% -> 60.4%) and Telebirr growth (27M -> 47M).
*   **New Events:** M-Pesa launch date anchored in August 2023.

**Next Step:** Task 2 (Exploratory Data Analysis) to visualize these trends and calculate correlations.